<a href="https://colab.research.google.com/github/conuwoo/AI-Camp-Summer-2022/blob/main/ML_Activity/Scratch_LogisticReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd

In [14]:
train_data = pd.read_csv("titanic_train.csv")
test_data = pd.read_csv("titanic_test.csv")

Data PreProcessing

In [15]:
train_data = train_data.drop(['Ticket'], axis=1)
test_data = test_data.drop(['Ticket'], axis=1)

In [16]:
train_data = train_data.drop(['Name'], axis=1)
test_data = test_data.drop(['Name'], axis=1)

Fill na values

In [17]:
#Check for na's
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [18]:
test_data.isna().sum()

PassengerId      0
Pclass           0
Sex              0
Age             86
SibSp            0
Parch            0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [19]:
train_data['Age'] = train_data['Age'].replace(np.nan,train_data['Age'].mean())
test_data['Age'] = test_data['Age'].replace(np.nan, test_data['Age'].mean())

In [20]:
#Shows there is more than half of NA's for the column so we should drop it
train_data = train_data.drop(['Cabin'], axis=1)
test_data = test_data.drop(['Cabin'], axis=1)

In [21]:
train_data['Embarked'] = train_data['Embarked'].replace(np.nan,'Q')

In [22]:
test_data['Fare'] = test_data['Fare'].replace(np.nan, test_data['Fare'].mean())

In [23]:
#Check for na's
train_data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [24]:
test_data.isna().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

Encode categorical variables

In [25]:
from sklearn.preprocessing import LabelEncoder

#initialize label endoce as first step.
le = LabelEncoder()

train_data["Sex"] = le.fit_transform(train_data["Sex"].values)
test_data["Sex"] = le.fit_transform(test_data["Sex"].values)

train_data["Embarked"] = le.fit_transform(train_data["Embarked"].values)
test_data["Embarked"] = le.fit_transform(test_data["Embarked"].values)

In [26]:
from sklearn.model_selection import train_test_split
features = ["Pclass","Sex","Age","SibSp","Fare"]
X = train_data[features]
y = train_data['Survived']

X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2)

from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()
X_train = min_max.fit_transform(X_train)
X_test = min_max.transform(X_test)

Logistic Regression

In [5]:
def sigmoid(z): # takes linear functinon "z" as an input

    g_function = 1/(1+np.exp(-z))
   
    return g_function

In [53]:
def cost(X, y, w, b):

    m = X.shape[0] # 0 to m-1
    cost = 0 # starts with 0
    for i in range(m): # sigma
        current_z = np.dot(X[i],w) + b # linear function "z"
        current_f = sigmoid(current_z) # input "z" into sigmoid function
        cost +=  -y[i]*np.log(current_f) - (1-y[i])*np.log(1-current_f) # logloss formula
             
    cost = cost / m # divide by m iterations
    return cost

In [42]:
X_train

array([[0.        , 0.        , 0.2963056 , 0.375     , 0.51334181],
       [1.        , 1.        , 0.24604172, 0.        , 0.01921772],
       [0.5       , 1.        , 0.42196532, 0.125     , 0.04098927],
       ...,
       [1.        , 0.        , 0.01985423, 0.        , 0.0239836 ],
       [1.        , 1.        , 0.59788892, 0.        , 0.01533038],
       [0.        , 1.        , 0.560191  , 0.125     , 0.16293235]])

In [32]:
w = np.zeros((5,), dtype=int)
print(w.shape)
b = 0

(5,)


In [46]:
y_train = y_train.to_numpy()

In [54]:
cost(X_train,y_train,w,b)

0.693147180559947

In [55]:
def gradient(X, y, w, b): 
    m = X.shape[0]
    n = X.shape[1]
    grad_w = np.zeros((n,)) # gradient for w                          
    grad_b = 0 # gradient for b

    for i in range(m):
        current_f = sigmoid(np.dot(X[i],w) + b)       #g(z)
        loss  = current_f  - y[i]                       #loss
        for j in range(n): # sigma
            grad_w[j] = grad_w[j] + loss * X[i,j]      #gradient for current w
        grad_b = grad_b + loss # graident for current b
    grad_w = grad_w/m      # sigma/m
    grad_b = grad_b/m      # sigma/m
        
    return grad_b, grad_w  

In [57]:
gradient(X_train,y_train,w,b)

(0.10533707865168539,
 array([ 0.13588483,  0.20014045,  0.0454676 ,  0.00851475, -0.00450982]))

In [58]:
def gradient_descent(X, y, w, b, alpha, num_iters): 
    for i in range(num_iters):
        # Calculate the gradient and update the parameters
        grad_b, grad_w = gradient(X, y, w, b)   

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * grad_w               
        b = b - alpha * grad_b              

        if i% 100 == 0:
          print('Iteration: ' + str(i) + ', Loss: ' + str(cost(X,y,w,b)))
        
    return w, b  #return w and b

In [59]:
gradient_descent(X_train,y_train,w,b,0.01,10000)

Iteration: 0, Loss: 0.6924310277023202
Iteration: 100, Loss: 0.6445228360554707
Iteration: 200, Loss: 0.6214712261815585
Iteration: 300, Loss: 0.607533188989344
Iteration: 400, Loss: 0.5972728631751179
Iteration: 500, Loss: 0.5887304744781059
Iteration: 600, Loss: 0.5811577741817543
Iteration: 700, Loss: 0.5742478320124913
Iteration: 800, Loss: 0.5678609524634668
Iteration: 900, Loss: 0.5619231989605004
Iteration: 1000, Loss: 0.556387671662148
Iteration: 1100, Loss: 0.5512193898234734
Iteration: 1200, Loss: 0.5463892720346941
Iteration: 1300, Loss: 0.5418716819384012
Iteration: 1400, Loss: 0.5376433865349867
Iteration: 1500, Loss: 0.5336830780175502
Iteration: 1600, Loss: 0.5299711216866183
Iteration: 1700, Loss: 0.5264893955152085
Iteration: 1800, Loss: 0.5232211680207065
Iteration: 1900, Loss: 0.5201509935372726
Iteration: 2000, Loss: 0.5172646168978146
Iteration: 2100, Loss: 0.5145488846018199
Iteration: 2200, Loss: 0.5119916614679568
Iteration: 2300, Loss: 0.5095817524477708
Iterat

(array([-1.3629786 , -2.43172339, -0.11372335, -0.215576  ,  0.50271047]),
 1.8934886014151175)